# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there if you'd like: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish as cf
>>> cf.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("/Volumes/SSD01/datastore/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
derivative_step = 0.01 #How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Fiducial Cosmology 

Here we specify the fiducial cosmology used in our forecasts. For light relic forecasts, we choose whether to vary with respect to the relic mass or the relic temperature, specified by the "relic_fix" parameter. Since we are interested in forecasting errors on T_chi, we choose to vary  T_ncdm. 

In [ ]:
relicfid = {
        "A_s" : 2.22e-9, 
        "n_s" : 0.965,
        "omega_b" : 0.02222,
        "omega_cdm" : 0.1120,
        "tau_reio" : 0.06,
        "h" : 0.70,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : (1.50/2.726), # Units [T_cmb]. We choose this temp, 1.5 K, because that's what our CMBS4 priors are calculated at.
        "m_ncdm" : 0.03, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_fix" : "m_ncdm" # Fix T_ncdm or m_ncdm 
        }

# Specify Experiment Observational Parameters

The experimental redshift configuration and noise is specified here. 

In [ ]:
#Specify redshift bins, noise per bin, and sky coverage
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence at one Fiducial Mass

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
relicconvergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'relic', # 'relic' or 'neutrino' forecasting scheme 
    relicfid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    mu_step=mu_integral_step,
    varyfactors=[0.001, 0.003, 0.005, 0.007, 0.009] # Relative factors used to compute convergence
    )
relicconvergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecasts

In [ ]:
masses = np.geomspace(0.1, 10.0, 21) 
omegacdm_set = relicfid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(1.5 / 1.95, 3.))
                                        
fiducialset = [dict(relicfid, **{'m_ncdm' : masses[midx], 'omega_cdm' : omegacdm_set[midx]}) 
               for midx, mval in enumerate(masses)]
 
forecastset = [cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) for fididx, fidval in enumerate(fiducialset)]

for fidx, fcst in enumerate(forecastset): 
    fcst.gen_pm()
    fcst.gen_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                          
            #'M_ncdm',                                    
            'T_ncdm',                                 
            'sigma_fog',                                   
            'b0',                                         
            'alpha_k2'],
        mu_step=mu_integral_step, 
        skipgen=False)
    print("Relic Forecast ", fidx, " complete...")

In [ ]:
inpath = "/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/cosmicfish/priors/CMBS4_Fisher_Relic_"
outpath = "/Users/nicholasdeporzio/Desktop/cfworkspace/results/scienceresults_1K/covariances/"

for fidx, fval in enumerate(forecastset):
    fval.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'T_ncdm'],
        fisherpath=inpath+str(fidx+1)+".dat")
    print("CMB Fisher information loaded to forecast " + str(fidx) + "...")
    fval.export_matrices("~/Desktop")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_cmbfisher.mat", outpath+"cmb/"+"inv_cmbfisher_"+str(fidx+1)+".mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_lssfisher.mat", outpath+"lss/"+"inv_lssfisher_"+str(fidx+1)+".mat")
    shutil.move("/Users/nicholasdeporzio/Desktop/inv_fullfisher.mat", outpath+"full/"+"inv_fullfisher_"+str(fidx+1)+".mat")

In [ ]:
#Save all the LSS/CMB/Full fisher matrices  
for fidx, fcst in enumerate(forecastset):
    name = '/Users/nicholasdeporzio/Desktop/cfworkspace/results/scienceresults_1K/fishers/full/FULL_Fisher_' + str(fidx+1) + '.dat'
    head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t T_ncdm \t sigma_fog \t b_0 \t alpha_k2'
    np.savetxt(name, fcst.numpy_full_fisher, delimiter='\t', header=head)

In [ ]:
errors_Tncdm_cmb = [np.sqrt(fval.numpy_cmb_covariance[6,6]) for fidx, fval in enumerate(forecastset)]    
errors_Tncdm_lss = [np.sqrt(fval.numpy_lss_covariance[6,6]) for fidx, fval in enumerate(forecastset)]
errors_Tncdm_full = [np.sqrt(fval.numpy_full_covariance[6,6]) for fidx, fval in enumerate(forecastset)]

plt.figure(figsize=(15,7.5))
plt.semilogx(masses, errors_Tncdm_full, label="CMB+LSS")
plt.semilogx(masses, errors_Tncdm_lss, label="LSS")
plt.semilogx(masses, errors_Tncdm_cmb, label="CMB")
plt.title("Relic Temperature Uncertainty")
plt.xlabel("Relic Mass [eV]")
plt.ylabel(r"$\sigma_T ~[K]$")
plt.legend()
plt.savefig("/Users/nicholasdeporzio/Desktop/results.png")
plt.show()

In [ ]:
import dill
dill.dump_session("/Users/nicholasdeporzio/Desktop/Relic_Science_Plot.db")